In [2]:
import re
from ugot import ugot
got = ugot.UGOT()
got.initialize('192.168.1.189')

def parse_command(speech):
    pattern = r"\b(forward|backward|left|right)\s+(\d+(?:\.\d+)?)\b"
    match = re.search(pattern, speech.lower())
    if match:
        return match.group(1), float(match.group(2))
    return None, None

def drive_with_speech():
    print("Voice-controlled driving started. Say 'exit' or 'quit' to stop.")
    while True:
        # Listen for up to 6 seconds
        _, speech = got.start_audio_asr_doa(duration=6)
        print(f"Recognized: {speech!r}")

        if not speech:
            got.play_audio_tts("I didn't hear anything. Please repeat.", 0, wait=True)
            continue

        # Exit condition
        if any(i in speech.lower() for i in ("exit", "quit")):
            got.play_audio_tts("Shutting down voice control. Goodbye!", 0, wait=True)
            break

        # Try to parse a movement command
        cmd, dist = parse_command(speech)
        steps = int(dist)
        if cmd:
            got.play_audio_tts(f"Moving {cmd} {steps} meters", 0, wait=True)
            if cmd == "forward":
                got.mecanum_move_speed_times(0, 30, steps, 1)
            elif cmd == "backward":
                got.mecanum_move_speed_times(1, 30, steps, 1)
            elif cmd == "left":
                got.mecanum_turn_speed_times(2, 30, steps, 2)
            elif cmd == "right":
                got.mecanum_turn_speed_times(3, 30, steps, 2)
        else:
            got.play_audio_tts(
                "Command not recognized. "
                "Please say something like 'forward 5' or 'left 2.5'.",
                0,
                wait=True
            )

drive_with_speech()


192.168.1.189:50051
Voice-controlled driving started. Say 'exit' or 'quit' to stop.
Recognized: 'Forward 10'
Recognized: 'Left 90'
Recognized: 'Backwards 10 left 90'
Recognized: 'Exit'


In [3]:
import re
from ugot import ugot

got = ugot.UGOT()
got.initialize('192.168.1.189')

def parse_commands(speech):
    pattern = r"\b(forward|backward|left|right)\s+(\d+(?:\.\d+)?)\b"
    matches = re.findall(pattern, speech.lower())
    return [(cmd, float(dist)) for cmd, dist in matches]

def drive_with_speech():
    print("Voice-controlled driving started. Say 'exit' or 'quit' to stop.")
    while True:
        # Listen for up to 6 seconds
        _, speech = got.start_audio_asr_doa(duration=6)
        print(f"Recognized: {speech!r}")

        if not speech:
            got.play_audio_tts("I didn't hear anything. Please repeat.", 0, wait=True)
            continue

        # Exit check
        if any(i in speech.lower() for i in ("exit", "quit")):
            got.play_audio_tts("Shutting down voice control. Goodbye!", 0, wait=True)
            break

        # Parse all movement commands
        commands = parse_commands(speech)
        if not commands:
            got.play_audio_tts(
                "Command not recognized. "
                "Please say something like 'forward 5 then left 2'.",
                0, wait=True
            )
            continue

        # Execute each in sequence
        for cmd, dist in commands:
            steps = int(dist)
            got.play_audio_tts(f"Moving {cmd} {dist} meters", 0, wait=True)

            if cmd == "forward":
                got.mecanum_move_speed_times(0, 30, steps, 1)
            elif cmd == "backward":
                got.mecanum_move_speed_times(1, 30, steps, 1)
            elif cmd == "left":
                got.mecanum_turn_speed_times(2, 30, steps, 2)
            elif cmd == "right":
                got.mecanum_turn_speed_times(3, 30, steps, 2)

drive_with_speech()


192.168.1.189:50051
Voice-controlled driving started. Say 'exit' or 'quit' to stop.
Recognized: 'Forward 10 backwards 10 left 90 right 90'
Recognized: 'Exit'


In [ ]:
import re
from ugot import ugot

got = ugot.UGOT()
got.initialize('192.168.1.189')

def parse_commands(speech):
    pattern = r"\b(forward|backward|left|right)\s+(\d+(?:\.\d+)?)\b"
    matches = re.findall(pattern, speech.lower())
    return [(cmd, float(dist)) for cmd, dist in matches]

def drive_with_speech():
    current_speed = 30
    print("Voice‑controlled driving started. Say 'exit' to stop or 'speed 50' to change speed.")
    while True:
        _, speech = got.start_audio_asr_doa(duration=6)
        print(f"Recognized: {speech!r}")

        if not speech:
            got.play_audio_tts("I didn't hear anything. Please repeat.", 0, wait=True)
            continue

        lower = speech.lower()

        # Exit if requested
        if any(i in lower for i in ("exit", "quit")):
            got.play_audio_tts("Shutting down voice control. Goodbye!", 0, wait=True)
            break

        # Check for speed‑setting phrases
        speed_matches = re.findall(r"\bspeed\s+(\d+)\b", lower)
        if speed_matches:
            current_speed = int(speed_matches[-1])
            got.play_audio_tts(f"Speed set to {current_speed}", 0, wait=True)
            lower = re.sub(r"\bspeed\s+\d+\b", "", lower)

        # Parse any movement commands
        commands = parse_commands(lower)
        if not commands:
            if speed_matches:
                continue
            got.play_audio_tts(
                "Command not recognized. "
                "Try 'forward 5', 'left 2', or 'speed 40'.",
                0, wait=True
            )
            continue

        # Execute each in sequence
        for cmd, dist in commands:
            steps = int(dist) 
            got.play_audio_tts(
                f"Moving {cmd} {dist} meters at speed {current_speed}",
                0, wait=True
            )

            if cmd == "forward":
                got.mecanum_move_speed_times(0, current_speed, steps, 1)
            elif cmd == "backward":
                got.mecanum_move_speed_times(1, current_speed, steps, 1)
            elif cmd == "left":
                got.mecanum_turn_speed_times(2, current_speed, steps, 2)
            elif cmd == "right":
                got.mecanum_turn_speed_times(3, current_speed, steps, 2)


drive_with_speech()
